In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from data import resample_nba_data as re
from data import clean_and_split_nba_data as clean
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from models import plot_validation_curve as vc
from models import eval_model as evm
from joblib import dump
import joblib
%load_ext autoreload
%autoreload 2

In [2]:
sp_wk3_linear_reg_model	 = joblib.load("../models/sp_wk3_linear_reg_model.joblib")
sp_wk3_logistic_reg_model = joblib.load("../models/sp_wk3_logistic_reg_model.joblib")
sp_wk3_rf_downsample_model = joblib.load("../models/sp_wk3_rf_downsample_model.joblib")
sp_wk3_rf_upsample_model = joblib.load("../models/sp_wk3_rf_upsample_model.joblib")
sp_wk3_xgboost_downsample_model = joblib.load("../models/sp_wk3_xgboost_downsample_model.joblib")
sp_wk3_xgboost_upsample_model = joblib.load("../models/sp_wk3_xgboost_upsample_model.joblib")
sp_wk3_kmean_model = joblib.load("../models/sp_wk3_kmean_model.joblib")
sp_wk3_gussian_mixture_model = joblib.load("../models/sp_wk3_gussian_mixture_model.joblib")

sp_wk3_Stg2_Final_model = joblib.load("../models/sp_wk3_Stg2_Final_model.joblib")


scaler = np.load('../models/sp_wk3_scaler_logistic_regression.joblib',allow_pickle=True)

In [3]:
df=pd.read_csv('../data/raw/test.csv')

In [4]:
def clean_nba_data(df):
    """
    Imports the file and splits it into Train, Valdiation and Test set.
    It returns all splits.
    
    Parameters
    ----------
    file : input csv file with the path
    """
    
    df_cleaned = df.copy()
    df_cleaned[ df_cleaned<0 ] = 0
    df_cleaned.loc[df_cleaned['3P Made'] > df_cleaned['3PA'], ['3P Made' , '3PA', 'CALC3P%']] = 0, 0, 0
    df_cleaned.loc[df_cleaned['FGM'] > df_cleaned['FGA'], ['FGM', 'FGA', 'CALCFG%']] = 0, 0, 0
    df_cleaned.loc[df_cleaned['FTM'] > df_cleaned['FTA'], ['FTM', 'FTA', 'CALCFT%']] = 0, 0, 0
    df_cleaned.loc[df_cleaned['3P Made'] > 0, ['CALC3P%']] = df_cleaned['3P Made']/df_cleaned['3PA']*100
    df_cleaned.loc[df_cleaned['FGM'] > 0, ['CALCFG%']] =df_cleaned['FGM']/df_cleaned['FGA']*100
    df_cleaned.loc[df_cleaned['FTM'] > 0, ['CALCFT%']] = df_cleaned['FTM']/df_cleaned['FTA']*100
    df_cleaned = df_cleaned.drop(['3P%','FT%','FG%','Id_old'],axis=1)
    df_cleaned = df_cleaned.fillna(0)
    return df_cleaned

In [5]:
x_data_stg1 = clean_nba_data(df)

In [6]:
df=x_data_stg1.copy()
x_data_stg1 = x_data_stg1.drop('Id',axis=1)

In [7]:
df

,Id,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,CALC3P%,CALCFG%,CALCFT%
0,0,56,9.1,4.0,1.6,3.7,0.1,0.3,0.7,1.2,1.2,0.8,1.7,0.4,0.2,0.3,0.8,33.333333,43.243243,58.333333
1,1,43,19.3,10.1,3.7,8.1,0.6,1.7,1.8,2.5,0.5,0.9,1.5,3.5,0.6,0.0,1.8,35.294118,45.679012,72.000000
2,2,82,33.9,11.3,4.9,10.6,0.5,1.9,1.8,2.7,1.3,3.3,4.5,2.5,1.3,0.3,2.0,26.315789,46.226415,66.666667
3,3,86,44.7,18.8,6.8,15.9,0.5,1.8,4.5,6.3,1.5,3.2,5.0,4.1,0.9,0.1,3.6,27.777778,42.767296,71.428571
4,4,58,12.3,4.7,1.6,4.0,0.5,1.7,1.1,1.3,0.2,0.6,0.9,1.5,0.5,0.0,0.9,29.411765,40.000000,84.615385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,3794,84,21.2,8.7,3.4,6.7,0.0,0.0,1.7,2.5,1.9,2.3,3.9,1.5,0.6,0.3,2.0,0.000000,50.746269,68.000000
3795,3795,49,16.3,6.4,2.9,6.6,0.0,0.0,1.0,1.9,1.7,2.8,4.4,0.4,0.4,0.4,0.7,0.000000,43.939394,52.631579
3796,3796,53,9.9,2.1,0.8,1.8,0.0,0.0,0.6,1.0,0.7,1.0,1.7,0.4,0.4,0.2,0.5,0.000000,44.444444,60.000000
3797,3797,89,38.3,14.5,5.4,11.8,0.5,1.2,2.5,2.9,1.5,4.0,5.5,3.7,1.3,0.3,2.4,41.666667,45.762712,86.206897


In [8]:
x_data_stg1.shape

(3799, 19)

In [9]:
x_data_stg1.columns

Index(['GP', 'MIN', 'PTS', 'FGM', 'FGA', '3P Made', '3PA', 'FTM', 'FTA',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'CALC3P%', 'CALCFG%',
       'CALCFT%'],
      dtype='object')

In [10]:
def _pred_prob_for_model(mod,df):
    y_pred=0.0
    y_pred=mod.predict_proba(df)
    print('Classes',mod.classes_)
#     print('Unique Values ',np.unique(y_pred,return_counts=True))
    print('Unique Values ',np.unique(y_pred.round(),return_counts=True))
    print('Unique Values for Prediction Class',np.unique(y_pred[:,1].round(),return_counts=True))
    return y_pred

# For GMM

In [11]:
y_pred_gmm = sp_wk3_gussian_mixture_model.predict(x_data_stg1)
y_pred_prob_gmm = sp_wk3_gussian_mixture_model.predict_proba(x_data_stg1)

In [12]:
print('Unique Values for Prediction Class',np.unique(y_pred_gmm,return_counts=True))

Unique Values for Prediction Class (array([0, 1], dtype=int64), array([2670, 1129], dtype=int64))


In [13]:
print('Unique Values for Prediction Class',np.unique(y_pred_prob_gmm[:,0].round(),return_counts=True))
print('Unique Values for Prediction Class',np.unique(y_pred_prob_gmm[:,1].round(),return_counts=True))

Unique Values for Prediction Class (array([0., 1.]), array([1129, 2670], dtype=int64))
Unique Values for Prediction Class (array([0., 1.]), array([2670, 1129], dtype=int64))


In [14]:
df['gmm_probability']=y_pred_prob_gmm[:,1]

# Logistic Regression

In [15]:
x_data_stg1_lir= scaler.fit_transform(x_data_stg1)

In [16]:
y_pred_log_reg = _pred_prob_for_model(sp_wk3_logistic_reg_model,x_data_stg1_lir)

Classes [0 1]
Unique Values  (array([0., 1.]), array([3799, 3799], dtype=int64))
Unique Values for Prediction Class (array([0., 1.]), array([1161, 2638], dtype=int64))


In [17]:
df['logistic_reg_probability']=y_pred_log_reg[:,1]

# Random Forest Down-Sample Prediction

In [18]:
y_pred_rf_ds = _pred_prob_for_model(sp_wk3_rf_downsample_model,x_data_stg1)

Classes [0 1]
Unique Values  (array([0., 1.]), array([3799, 3799], dtype=int64))
Unique Values for Prediction Class (array([0., 1.]), array([1513, 2286], dtype=int64))


In [19]:
df['rf_ds_probability']=y_pred_rf_ds[:,1]

# Random Forest Up-Sample Prediction

In [20]:
y_pred_rf_us = _pred_prob_for_model(sp_wk3_rf_upsample_model,x_data_stg1)

Classes [0 1]
Unique Values  (array([0., 1.]), array([3799, 3799], dtype=int64))
Unique Values for Prediction Class (array([0., 1.]), array([1543, 2256], dtype=int64))


In [21]:
df['rf_us_probability']=y_pred_rf_us[:,1]

# XGBoost Down-Sample Prediction

In [22]:
y_pred_xg_ds = _pred_prob_for_model(sp_wk3_xgboost_downsample_model,x_data_stg1)

Classes [0 1]
Unique Values  (array([0., 1.], dtype=float32), array([3799, 3799], dtype=int64))
Unique Values for Prediction Class (array([0., 1.], dtype=float32), array([1492, 2307], dtype=int64))


In [23]:
df['xg_ds_probability']=y_pred_xg_ds[:,1]

# XGBoost Up-Sample Prediction

In [24]:
y_pred_xg_us = _pred_prob_for_model(sp_wk3_xgboost_upsample_model,x_data_stg1)

Classes [0 1]
Unique Values  (array([0., 1.], dtype=float32), array([3799, 3799], dtype=int64))
Unique Values for Prediction Class (array([0., 1.], dtype=float32), array([1562, 2237], dtype=int64))


In [25]:
df['xg_us_probability']=y_pred_xg_us[:,1]

In [26]:
df

,Id,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,...,TOV,CALC3P%,CALCFG%,CALCFT%,gmm_probability,logistic_reg_probability,rf_ds_probability,rf_us_probability,xg_ds_probability,xg_us_probability
0,0,56,9.1,4.0,1.6,3.7,0.1,0.3,0.7,1.2,...,0.8,33.333333,43.243243,58.333333,0.0,0.497949,0.368252,0.414024,0.359396,0.327457
1,1,43,19.3,10.1,3.7,8.1,0.6,1.7,1.8,2.5,...,1.8,35.294118,45.679012,72.000000,0.0,0.504697,0.411084,0.513047,0.489886,0.493140
2,2,82,33.9,11.3,4.9,10.6,0.5,1.9,1.8,2.7,...,2.0,26.315789,46.226415,66.666667,0.0,0.514147,0.637954,0.584404,0.676121,0.673573
3,3,86,44.7,18.8,6.8,15.9,0.5,1.8,4.5,6.3,...,3.6,27.777778,42.767296,71.428571,0.0,0.524392,0.637954,0.574986,0.627141,0.623453
4,4,58,12.3,4.7,1.6,4.0,0.5,1.7,1.1,1.3,...,0.9,29.411765,40.000000,84.615385,0.0,0.498235,0.389956,0.425121,0.387918,0.404849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,3794,84,21.2,8.7,3.4,6.7,0.0,0.0,1.7,2.5,...,2.0,0.000000,50.746269,68.000000,1.0,0.510184,0.637954,0.583927,0.676121,0.673573
3795,3795,49,16.3,6.4,2.9,6.6,0.0,0.0,1.0,1.9,...,0.7,0.000000,43.939394,52.631579,1.0,0.503224,0.407351,0.543942,0.485590,0.474013
3796,3796,53,9.9,2.1,0.8,1.8,0.0,0.0,0.6,1.0,...,0.5,0.000000,44.444444,60.000000,1.0,0.496257,0.368252,0.423017,0.373433,0.377646
3797,3797,89,38.3,14.5,5.4,11.8,0.5,1.2,2.5,2.9,...,2.4,41.666667,45.762712,86.206897,0.0,0.518714,0.637954,0.586515,0.676121,0.673573


In [35]:
df.loc[:,df.columns!= 'Id']

,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,OREB,...,TOV,CALC3P%,CALCFG%,CALCFT%,gmm_probability,logistic_reg_probability,rf_ds_probability,rf_us_probability,xg_ds_probability,xg_us_probability
0,56,9.1,4.0,1.6,3.7,0.1,0.3,0.7,1.2,1.2,...,0.8,33.333333,43.243243,58.333333,0.0,0.497949,0.368252,0.414024,0.359396,0.327457
1,43,19.3,10.1,3.7,8.1,0.6,1.7,1.8,2.5,0.5,...,1.8,35.294118,45.679012,72.000000,0.0,0.504697,0.411084,0.513047,0.489886,0.493140
2,82,33.9,11.3,4.9,10.6,0.5,1.9,1.8,2.7,1.3,...,2.0,26.315789,46.226415,66.666667,0.0,0.514147,0.637954,0.584404,0.676121,0.673573
3,86,44.7,18.8,6.8,15.9,0.5,1.8,4.5,6.3,1.5,...,3.6,27.777778,42.767296,71.428571,0.0,0.524392,0.637954,0.574986,0.627141,0.623453
4,58,12.3,4.7,1.6,4.0,0.5,1.7,1.1,1.3,0.2,...,0.9,29.411765,40.000000,84.615385,0.0,0.498235,0.389956,0.425121,0.387918,0.404849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,84,21.2,8.7,3.4,6.7,0.0,0.0,1.7,2.5,1.9,...,2.0,0.000000,50.746269,68.000000,1.0,0.510184,0.637954,0.583927,0.676121,0.673573
3795,49,16.3,6.4,2.9,6.6,0.0,0.0,1.0,1.9,1.7,...,0.7,0.000000,43.939394,52.631579,1.0,0.503224,0.407351,0.543942,0.485590,0.474013
3796,53,9.9,2.1,0.8,1.8,0.0,0.0,0.6,1.0,0.7,...,0.5,0.000000,44.444444,60.000000,1.0,0.496257,0.368252,0.423017,0.373433,0.377646
3797,89,38.3,14.5,5.4,11.8,0.5,1.2,2.5,2.9,1.5,...,2.4,41.666667,45.762712,86.206897,0.0,0.518714,0.637954,0.586515,0.676121,0.673573


# Stg 2 Prediction

In [36]:
y_pred_xg_final = _pred_prob_for_model(sp_wk3_Stg2_Final_model,df.loc[:,df.columns!= 'Id'])

Classes [0 1]
Unique Values  (array([0., 1.], dtype=float32), array([3799, 3799], dtype=int64))
Unique Values for Prediction Class (array([0., 1.], dtype=float32), array([1883, 1916], dtype=int64))


In [40]:
df['TARGET_5Yrs']=y_pred_xg_final[:,1]

In [41]:
df

,Id,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,...,CALCFG%,CALCFT%,gmm_probability,logistic_reg_probability,rf_ds_probability,rf_us_probability,xg_ds_probability,xg_us_probability,final_probability,TARGET_5Yrs
0,0,56,9.1,4.0,1.6,3.7,0.1,0.3,0.7,1.2,...,43.243243,58.333333,0.0,0.497949,0.368252,0.414024,0.359396,0.327457,0.235432,0.235432
1,1,43,19.3,10.1,3.7,8.1,0.6,1.7,1.8,2.5,...,45.679012,72.000000,0.0,0.504697,0.411084,0.513047,0.489886,0.493140,0.413014,0.413014
2,2,82,33.9,11.3,4.9,10.6,0.5,1.9,1.8,2.7,...,46.226415,66.666667,0.0,0.514147,0.637954,0.584404,0.676121,0.673573,0.467124,0.467124
3,3,86,44.7,18.8,6.8,15.9,0.5,1.8,4.5,6.3,...,42.767296,71.428571,0.0,0.524392,0.637954,0.574986,0.627141,0.623453,0.850676,0.850676
4,4,58,12.3,4.7,1.6,4.0,0.5,1.7,1.1,1.3,...,40.000000,84.615385,0.0,0.498235,0.389956,0.425121,0.387918,0.404849,0.639221,0.639221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,3794,84,21.2,8.7,3.4,6.7,0.0,0.0,1.7,2.5,...,50.746269,68.000000,1.0,0.510184,0.637954,0.583927,0.676121,0.673573,0.902589,0.902589
3795,3795,49,16.3,6.4,2.9,6.6,0.0,0.0,1.0,1.9,...,43.939394,52.631579,1.0,0.503224,0.407351,0.543942,0.485590,0.474013,0.266726,0.266726
3796,3796,53,9.9,2.1,0.8,1.8,0.0,0.0,0.6,1.0,...,44.444444,60.000000,1.0,0.496257,0.368252,0.423017,0.373433,0.377646,0.433543,0.433543
3797,3797,89,38.3,14.5,5.4,11.8,0.5,1.2,2.5,2.9,...,45.762712,86.206897,0.0,0.518714,0.637954,0.586515,0.676121,0.673573,0.340340,0.340340


In [42]:
df.to_csv('../data/processed/Test_Stg1_Output.csv',index=False,columns=['Id', 'TARGET_5Yrs'])